In [2]:
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

import tensorflow as tf

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vineetsharma9252/dataset-for-research2")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dataset-for-research2


In [4]:
df = pd.read_csv("/kaggle/input/dataset-for-research2/dataset_for_research2.csv")

In [5]:
df.head(5)

,Unnamed: 0,source_file,date,time,message,drone_model_y,dataset_y,controller_y,words,labels,labels_encoded,Pos
0,0,336-80799827e1156d2c54438d9e04b6100afbaad8e3,6/29/2017,15:24:31,Taking Off.,DJI_Inspire_1,df010,mobile_iOS,Taking,B-ACTION,0,VERB
1,1,336-80799827e1156d2c54438d9e04b6100afbaad8e3,6/29/2017,15:24:31,Taking Off.,DJI_Inspire_1,df010,mobile_iOS,Off,I-ACTION,1,ADP
2,2,336-80799827e1156d2c54438d9e04b6100afbaad8e3,6/29/2017,15:24:31,Taking Off.,DJI_Inspire_1,df010,mobile_iOS,.,O,2,PUNCT
3,3,336-80799827e1156d2c54438d9e04b6100afbaad8e3,6/29/2017,15:24:31,Taking Off.,DJI_Inspire_1,df010,mobile_iOS,Taking,B-ACTION,0,VERB
4,4,336-80799827e1156d2c54438d9e04b6100afbaad8e3,6/29/2017,15:24:31,Taking Off.,DJI_Inspire_1,df010,mobile_iOS,Off,I-ACTION,1,ADP


In [6]:
df.labels.unique()

array(['B-ACTION', 'I-ACTION', 'O', 'B-PARAMETER', 'I-PARAMETER',
       'B-STATE', 'I-STATE', 'B-COMPONENT', 'B-ISSUE', 'I-ISSUE',
       'B-FUNCTION', 'I-FUNCTION', 'I-COMPONENT'], dtype=object)

In [7]:
!pip install datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [8]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification, TrainingArguments, Trainer

from datasets import Dataset

from sklearn.metrics import accuracy_score, f1_score

import numpy as np

import pandas as pd



class Alberta:

    def __init__(self):

        self.tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')



    def dataset_setting(self, dataset_text_att, dataset_label_att):

        text = dataset_text_att.tolist()

        labels = dataset_label_att.tolist()

        data = {"text": text, "labels": labels}

        dataset = Dataset.from_dict(data)

        dataset = dataset.train_test_split(test_size=0.2)

        return dataset



    def tokenize_function(self, examples):

        return self.tokenizer(examples['text'], padding='max_length', truncation=True)



    def mapping(self, dataset):

        tokenized_dataset = dataset.map(self.tokenize_function, batched=True)

        return tokenized_dataset



    def setting_format(self, columns, tokenized_dataset):

        tokenized_dataset.set_format(type='torch', columns=columns)

        return tokenized_dataset



    def init_model(self, num_labels=2):

        model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=num_labels)

        return model



    def training_args(self, output_dir, evaluation_strategy, learning_rate,

                      per_device_train_batch_size, per_device_eval_batch_size,

                      num_train_epochs, weight_decay):

        training_args = TrainingArguments(

            output_dir=output_dir,

            evaluation_strategy=evaluation_strategy,

            learning_rate=learning_rate,

            per_device_train_batch_size=per_device_train_batch_size,

            per_device_eval_batch_size=per_device_eval_batch_size,

            num_train_epochs=num_train_epochs,

            weight_decay=weight_decay,

        )

        return training_args



    def compute_metrics(self, eval_pred):

        logits, labels = eval_pred

        predictions = np.argmax(logits, axis=1)

        accuracy = accuracy_score(labels, predictions)

        f1 = f1_score(labels, predictions, average='weighted')

        return {"accuracy": accuracy, "f1": f1}



    def trainer_setup(self, model, training_args, train_dataset, eval_dataset):

        trainer = Trainer(

            model=model,

            args=training_args,

            train_dataset=train_dataset,

            eval_dataset=eval_dataset,

            tokenizer=self.tokenizer,

            compute_metrics=self.compute_metrics  # Add compute_metrics function here

        )

        return trainer



    def train(self, trainer):

        print("Training has been initiated")

        trainer.train()

        print("Training Completed")



    def result(self, trainer):
        print(trainer.evaluate())

    
    def classification_report(self, trainer, eval_dataset):
        # Make predictions
        predictions = trainer.predict(eval_dataset)
        logits, labels = predictions.predictions, predictions.label_ids
        
        # Get the predicted classes
        preds = np.argmax(logits, axis=1)
        
        # Generate the classification report
        report = classification_report(labels, preds, target_names=['B-ACTION', 'I-ACTION', 'O', 'B-PARAMETER', 'I-PARAMETER',
       'B-STATE', 'I-STATE', 'B-COMPONENT', 'B-ISSUE', 'I-ISSUE',
       'B-FUNCTION', 'I-FUNCTION', 'I-COMPONENT'])
        return report


In [8]:
df.message = df.message.astype(str)

In [ ]:
# Dataset setup
alberta = Alberta()

dataset = alberta.dataset_setting(df['words'], df['labels_encoded'])

tokenized_dataset = alberta.mapping(dataset)
# Tokenize each split separately
from imblearn.over_sampling import RandomOverSampler
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np

# Step 1: Extract Training Data
train_df = tokenized_dataset['train'].to_pandas()

# Step 2: Apply Oversampling
ros = RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(
    np.array(train_df['text']).reshape(-1, 1),
    train_df['labels']
)

# Convert to pandas DataFrame
oversampled_train_df = pd.DataFrame({
    "text": x_resampled.ravel(),
    "labels": y_resampled
})

# Step 3: Convert Back to Dataset
oversampled_train_dataset = Dataset.from_pandas(oversampled_train_df)

# Step 4: Replace Training Split in DatasetDict
tokenized_dataset = DatasetDict({
    "train": oversampled_train_dataset,
    "test": dataset['test']
})

# Step 5: Tokenize the Dataset
tokenized_dataset = tokenized_dataset.map(alberta.tokenize_function, batched=True)




# Explicitly select 'train' and 'test' splits for further processing

# Format tokenized datasets
formatted_train = alberta.setting_format(['input_ids', 'attention_mask', 'labels'], tokenized_dataset['train'])
formatted_test = alberta.setting_format(['input_ids', 'attention_mask', 'labels'], tokenized_dataset['test'])

# Initialize model
model = alberta.init_model(num_labels=13)
model.to("cuda")

# Define training arguments
training_args = alberta.training_args(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=3,
    weight_decay=0.02,
)

# Setup trainer
trainer = alberta.trainer_setup(
    model=model,
    training_args=training_args,
    train_dataset=formatted_train,
    eval_dataset=formatted_test
)

# Train the model
alberta.train(trainer)

# Evaluate and generate classification report
alberta.result(trainer)
report = alberta.classification_report(trainer, formatted_test)
print(report)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/7282 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Map:   0%|          | 0/51168 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training has been initiated


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mastergenos228 (mastergenos228-government-engineering-college-ajmer). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


In [ ]:
!nvidia-smi

In [15]:
dataset = alberta.dataset_setting(df['words'], df['labels_encoded'])


# Tokenize each split separately
tokenized_dataset = alberta.mapping(dataset)

# Explicitly select 'train' and 'test' splits for further processing
train_split = tokenized_dataset['train']
test_split = tokenized_dataset['test']

Map:   0%|          | 0/7282 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [16]:
train_split

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7282
})

In [17]:
test_split

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1821
})

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7282
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [19]:
from imblearn.over_sampling import RandomOverSampler
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np

# Example DatasetDict (replace with your actual dataset)
# dataset = DatasetDict({
#     "train": train_dataset,
#     "test": test_dataset
# })

# Step 1: Extract Training Data
train_df = dataset['train'].to_pandas()

# Step 2: Apply Oversampling
ros = RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(
    np.array(train_df['text']).reshape(-1, 1),
    train_df['labels']
)

# Convert to pandas DataFrame
oversampled_train_df = pd.DataFrame({
    "text": x_resampled.ravel(),
    "labels": y_resampled
})

# Step 3: Convert Back to Dataset
oversampled_train_dataset = Dataset.from_pandas(oversampled_train_df)

# Step 4: Replace Training Split in DatasetDict
dataset = DatasetDict({
    "train": oversampled_train_dataset,
    "test": dataset['test']
})

# Step 5: Tokenize the Dataset
dataset = dataset.map(alberta.tokenize_function, batched=True)

# Step 6: Set Format for PyTorch
dataset['train'].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
dataset['test'].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/50817 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50817
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})